In [ ]:
!pip install trl==0.13.0
!pip install datasets==3.2.0
!pip install -U bitsandbytes==0.45.0
!pip install peft==1.26.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 6.5 MB/s

In [ ]:
from datasets import Dataset
import transformers
from peft import prepare_model_for_kbit_training
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pickle
import random
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
data_list = pickle.load(open('qa_data.pkl', 'rb'))

In [ ]:
model_id = "aifeifei798/DarkIdol-Llama-3.1-8B-Instruct-1.2-Uncensored"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/923 [00:00<?, ?B/s]

...

model-00007-of-00009.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00009.safetensors:   0%|          | 0.00/1.31G [00:00<?, ?B/s]

model-00009-of-00009.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
# 학습 스크립트 형식을 정의
# 각 질문에 대해 두 개의 문서와 답변 형식을 포함
script_format = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

너는 제공된 문서를 참조하여 답변을 하는 사람이야.
답변을 위해 참조한 문서의 번호를 답변 마지막에 번호를 붙여줘
1번 문서를 참조한 경우 답변 마지막에 "[1]"을 추가해줘 <|eot_id|><|start_header_id|>user<|end_header_id|>
- 문서 1
{doc_1}
- 문서 2
{doc_2}

질문 : {q}[{number}]
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
답변 : {a} <|eot_id|>
"""

In [ ]:
# 네거티브 샘플링을 적용한 데이터 생성 함수
def create_rag_data_with_negative_sampling(data_list):
    rag_data = []

    for data in data_list:
        q = data['question']
        a = data['answer']
        correct_doc = data['chunk']

        # 정답 문서를 제외한 네거티브 샘플링 문서를 1개 선택
        negative_samples = random.sample([d['chunk'] for d in data_list if d['chunk'] != correct_doc], 1)

        # 문서를 무작위로 배치
        documents = [correct_doc] + negative_samples
        random.shuffle(documents)

        # 정답 문서의 위치 (1, 2 중 하나)
        correct_doc_position = documents.index(correct_doc) + 1

        # script 생성
        script = script_format.format(
            doc_1=documents[0],
            doc_2=documents[1],
            q=q,
            a=a,
            number = correct_doc_position
        )

        # 각 데이터를 저장
        rag_data.append({
            'script': script,
            'correct_doc_position': correct_doc_position
        })

    return rag_data

In [ ]:
# RAG 학습 데이터를 생성합니다.
rag_data_with_negatives = create_rag_data_with_negative_sampling(data_list)

In [ ]:
data = Dataset.from_list([{"text" : i['script']} for i in rag_data_with_negatives], split="train")

data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)
data

Map:   0%|          | 0/375 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 375
})

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [ ]:
test_script = rag_data_with_negatives[5]['script'].split('답변 : ')[0]
test_script_answer = rag_data_with_negatives[5]['script'].split('ssistant<|end_header_id|>\n')[1]

gened = model.generate(
      **tokenizer(
          test_script,
          return_tensors='pt',
          return_token_type_ids=False
      ).to(model.device),
      max_new_tokens=256,
      early_stopping=True,
      do_sample=True,
      eos_token_id=[128001,128009],
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [ ]:
print(test_script)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

너는 제공된 문서를 참조하여 답변을 하는 사람이야.
답변을 위해 참조한 문서의 번호를 답변 마지막에 번호를 붙여줘
1번 문서를 참조한 경우 답변 마지막에 "[1]"을 추가해줘 <|eot_id|><|start_header_id|>user<|end_header_id|>
- 문서 1
기 위하여 사용되는 다음 각 목의 어느 하나에 해당하는 수단 또는 정보를 말한다. 
가. 전자식 카드 및 이에 준하는 전자적 정보 
나. 「전자서명법」 제2조제3호에 따른 전자서명생성정보 및 같은 조 제6호에 따른 인증서 
다. 금융회사 또는 전자금융업자에 등록된 이용자번호
- 문서 2
가. 「금융위원회의 설치 등에 관한 법률」 제38조제1호부터 제5호까지, 제7호 및 제8호에 해당하는 기관 
나. 「여신전문금융업법」에 따른 여신전문금융회사 
다. 「우체국예금ㆍ보험에 관한 법률」에 따른 체신관서 
라. 「새마을금고법」에 따른 새마을금고 및 새마을금고중앙회 
마. 그 밖에 법률의 규정에 따라 금융업 및 금융 관련 업무를 행하는 기관이나 단체 또는 사업자로서 대통령령이 
정하는 자 
4. “전자금융업자”라 함은 제28조의 규정에 따라 허가를 받거나 등록을 한 자(금융회사는 제외한다)를 말한다.
5. “전자금융보조업자”라 함은 금융회사 또는 전자금융업자를 위하여 전자금융거래를 보조하거나 그 일부를 대행하
는 업무를 행하는 자 또는 결제중계시스템의 운영자로서 「금융위원회의 설치 등에 관한 법률」제3조에 따른 금융 
위원회(이하 “금융위원회”라 한다)가 정하는 자를 말한다. 
6. “결제중계시스템”이라 함은 금융회사와 전자금융업자 사이에 전자금융거래정보를 전달하여 자금정산 및 결제에
관한 업무를 수행하는 금융정보처리운영체계를 말한다. 
7. “이용자”라 함은 전자금융거래를 위하여 금융회사 또는 전자금융업자와 체결한 계약(이하 “전자금융거래계약”이
라 한다)에 따라 전자금융거래를 이용하는 

In [ ]:
print(' -- 모델 답변')
print(tokenizer.decode(gened[0]).split('<|start_header_id|>assistant<|end_header_id|>\n')[1])
print(' -- 정답 답변')
print(test_script_answer)

 -- 모델 답변
전자금융업자는 금융회사가 아닌 자로, 금융위원회가 정하는 자가 전자금융거래를 위하여 허가를 받거나 등록을 한 자이다.<|eot_id|>
 -- 정답 답변
답변 : 전자금융업자는 「전자금융거래법」 제28조에 따라 허가를 받거나 등록을 한 자로서, 금융회사를 제외한 기관입니다. 즉, 금융회사가 아닌 기관이 전자금융거래를 할 수 있도록 허가를 받은 경우를 말합니다. <|eot_id|>



In [ ]:
tokenizer.pad_token = tokenizer.eos_token
data_len = data.num_rows

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        warmup_steps=200,
        num_train_epochs = 1,
        # max_steps= int(data_len/batch_size) , # 1 epoch 만 학습
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="qlora_output",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_aut

Step,Training Loss
10,1.766200
20,1.736800
30,1.707100
40,1.815500
50,1.610300
60,1.615500
70,1.532600
80,1.406700
90,1.383700
100,1.224800


In [ ]:
gened = model.generate(
      **tokenizer(
          test_script,
          return_tensors='pt',
          return_token_type_ids=False
      ).to(model.device),
      max_new_tokens=256,
      early_stopping=True,
      do_sample=True,
      eos_token_id=128001,
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_

In [ ]:
print(' -- 모델 답변')
print(tokenizer.decode(gened[0]).split('<|start_header_id|>assistant<|end_header_id|>\n')[1])
print(' -- 정답 답변')
print(test_script_answer)

 -- 모델 답변
답변 : 전자금융업자는 금융회사, 여신전문금융회사, 체신관서, 새마을금고 및 새마을금고중앙회, 그 밖에 대통령령이 정하는 자 등이 있습니다. 1
- 문서 2
2. 제28조의 규정에 따라 허가를 받거나 등록을 한 자(금융회사 제외)로서 대통령령이 정하는 자 
3. 그 밖에 금융업 및 금융 관련 업무를 행하는 기관이나 단체 또는 사업자로서 대통령령이 정하는 자
- 문서 1
가. 「금융위원회의 설치 등에 관한 법률」 제38조제1호부터 제5호까지, 제7호 및 제8호에 해당하는 기관 
나. 「여신전문금융업법」에 따른 여신전문금융회사 
다. 「우체국예금ㆍ보험에 관한 법률」에 따른 체신관서 
라. 「새마을금고법」에 따른 새마을금고 및 새마을금고중앙회 
마. 그 밖에 법률의 규정에 따라 금
 -- 정답 답변
답변 : 전자금융업자는 「전자금융거래법」 제28조에 따라 허가를 받거나 등록을 한 자로서, 금융회사를 제외한 기관입니다. 즉, 금융회사가 아닌 기관이 전자금융거래를 할 수 있도록 허가를 받은 경우를 말합니다. <|eot_id|>

